In [6]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
# -----------------------------
# 1) 데이터 준비
# -----------------------------
df = pd.read_csv("/content/drive/MyDrive/ml-programming-lab/week5/diabetes.csv")

# X, y 분리 (Outcome, BMI 제거 → X / BMI → y)
X = df.drop(columns=["Outcome", "BMI"]).values.astype("float32")
y = df["BMI"].values.astype("float32")

# 학습/테스트 분리 (random_state 제거하고 싶으면 해당 인자만 지워도 됨)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=24
)

# 스케일링(필수 권장): train으로 fit → test에 transform만
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [8]:
# -----------------------------
# 2) 모델 구성(MLP)
# -----------------------------
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.15),
    layers.Dense(64, activation="relu"),
    layers.Dense(1)  # 회귀는 선형 출력
])

model.compile(
    optimizer="adam",
    loss="mse",                        # 회귀 손실
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae"]
)

In [9]:
# -----------------------------
# 3) 학습
# -----------------------------
es = EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=300,
    batch_size=32,
    callbacks=[es],
    verbose=0
)

In [11]:
# -----------------------------
# 4) 평가
# -----------------------------
pred = model.predict(X_test, verbose=0).ravel()

mse  = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
mae  = mean_absolute_error(y_test, pred)
r2   = r2_score(y_test, pred)

print("=== Deep Learning (MLP) - BMI Regression ===")
print(f"Test MSE : {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE : {mae:.4f}")
print(f"Test R^2 : {r2:.4f}")

# 예측 샘플 5개
for i in range(5):
    print(f"[{i}] y_true={y_test[i]:.2f} | y_pred={pred[i]:.2f}")

=== Deep Learning (MLP) - BMI Regression ===
Test MSE : 57.9243
Test RMSE: 7.6108
Test MAE : 5.3319
Test R^2 : 0.2171
[0] y_true=29.50 | y_pred=23.54
[1] y_true=49.70 | y_pred=45.77
[2] y_true=27.40 | y_pred=31.62
[3] y_true=23.90 | y_pred=25.58
[4] y_true=42.80 | y_pred=48.71
